# Basic model of perceptual inference
* Basic implementation of Jazayeri and Movshon, 2006 Nature Neuroscience

* The model  addresses the issue of how to combine the output of multiple sensory neurons in order to infer the most likely state of a stimulus that is driving the observed pattern of responses across the entire population of cells. This is a classic problem in sensory neuroscience (or really any area where you're combining noisy/ambiguous signals to make an optimal inference). The reason the problem is so interesting is that the output of a single neuron is almost useless for performing inference, even if the neuron has a highly stable output (e.g. an orientation tuning function) that can be robustly measured. This ambiguity arises for a few reasons. 

* First, there is variability (the unpredictable kind, i.e. 'noise') in the output of neurons - so a response of 50Hz might be observed to stimulus 1 on trial 1, but a response of 40Hz might be observed on trial 2, etc. Second, the tuning function of most sensory neurons is non-monotonic (e.g. Gaussian-ish), so that almost all repsonse states are consistent with at least two possible stimuli (even in the complete absence of noise). Thus, a single measurement from a single neuron cannot be used to reliably transmit much information at all about the stimulus that was most likely to have caused a response. 

* Instead, inference about sensory stimuli is thought to be based on the output of many sensory neurons that are tuned to different points across a given feature space. For example, if you have just two neurons, and they are tuned to different points in feature space, it immediately becomes much easier to discriminate which feature was presented based on the output PATTERN across the two neurons. The more neurons you add (generally speaking) the more accurate your inference will be in the presence of noise because each neuron will contribute a little bit to disambiguating the input feature. 

* Note also that this approach generates a full estimate of the likelihood function, not just a point estimate of the most likely stimulus. This is a key feature of the approach, because different tasks require the implementation of different decision rules. For example, you might choose the most likely stimulus in the context of a discrimination, but you would want to compare the likelihood to some adjustable criterion. Furthermore, if you want to integrate multiple sources of information, such as multisensory integration or combining sensory evidence with a prior, then you need to deal in likelihood functions, not point estimates. This is a really critical advantage of this model over other approaches that optimally (i.e. use all available information) to determine just a point estimate of the most likely stimulus. 

## Imports

In [ ]:
import numpy as np
from numpy import pi as pi
import matplotlib.pyplot as plt

# also define the default font we'll use for figures. 
fig_font = {'fontname':'Arial', 'size':'20'}

## First some stimulus parameters - just going to simulate motion is different directions (i.e. a random dot stimulus ala Newsome). 

In [ ]:
# this is the direction of the stimulus that we'll try to decode
s1 = pi             

## Then set up some params that define the stimulus space (i.e. not the stim that we're presenting per se, but the space of all possible stimuli that we might evaluate)

In [ ]:
# number of possible directions
nDirs = 72   

# resolution sampling across direction space (radians)
dirStepSize = (2*pi)/nDirs             

# direction of each stimulus
sDir = np.linspace(0, (2*pi)-dirStepSize, nDirs)  

# xaxis of possible directions for plotting (Degrees)
xAxis = np.arange(0,360,360/nDirs)                

## Then some params to define our population of neurons
* Define the number of cells - we can play with this later
* Define the step size between the peaks of the tuning functions and then compute the actual peak of each tuning curve.
* Then define the 'concentration parameter' of each tuning function (the width of the circular Gaussian TF that we'll use)
* Want noise?

In [ ]:
# number of neurons
N = 72          

# resolution sampling across direction space (radians)
stimStepSize = (2*pi)/nDirs  

# set up the direction preference of each neuron
nDir = np.linspace(0, (2*pi)-2*pi/N, N) 

# then some other factors - tuning function width and any noise
# we want to add to the responses
k = 7            # concentration param for tuning curve 
noise = .05        # noise (IID) of neural responses

## Then build the actual tuning function for each neuron
* Use a circular Gaussian (Von Mises function)

In [ ]:
# loop over all possible sigs
fx = np.zeros((N,nDirs))

# loop over neurons...each of which is tuned to nDir[i]
for i in np.arange(N): 
    fx[i,:] = np.exp(k*(np.cos(sDir-nDir[i])-1))
                     
# set up a plotting axis in degrees (from radians)
xaxis = sDir*(180/pi)
plt.plot(xaxis, fx[0,])
plt.plot(xaxis, fx[36,])
plt.xlabel('Motion Direction', **fig_font)
plt.ylabel('Response', **fig_font)
plt.show()

# look at the full profile...
plt.imshow(fx, cmap='inferno', extent=[0, 360, 0, N])  # 'viridis', 'plasma', 'inferno', 'magma', etc
plt.xlabel('Motion Direction', **fig_font)
plt.ylabel('Neuron Number', **fig_font)
plt.show()

In [ ]:
fx.shape

## Figures 1-2 from J&M 2006 (and quick demo of LaTeX)
* Refer to equation 4 from the paper:


$$\log L(\theta) = \sum_{i=1}^{N}n_{i}{\cos(\theta - \theta_{i})}$$


* This is a rewriting of equ 3, where you had:

$$\log L(\theta) = \sum_{i=1}^{N}n_{i}{\log f_{i}(\theta)}$$
 
* So if: 

$$f(\theta) = e^{x}$$

* then: 

$$\log(e^{x}) = x$$
    
* which gives equation 4:

$$\log L(\theta) = \sum_{i=1}^{N}n_{i}{\cos(\theta - \theta_{i})}$$

### Given our bank of sensory neurons with tuning functions, generate a likelihood function to eval the most likely stim given pattern of observed responses
* First, we'll get the response vector to the stimulus of interest this R vector will correspond to the n1...ni vector in the paper
* Bonus - what is wrong with this version of the code??? 
    * re-run this cell several times with non-zero noise...what happens?
    * what is the problem and how to we fix it?

In [ ]:
sInd = np.argmin(np.abs(s1-sDir))

# pull a column out of the matrix that defines our TFs. 
R = fx[:,sInd]

# add noise if desired
print('Noise: ', noise)
if noise:
    R += (np.random.randn(len(R)) * noise)   

# plot
plt.plot(R, 'k-', linewidth=2)
plt.xlabel('Neuron Number', **fig_font)
plt.ylabel('Response', **fig_font)
plt.show()

### What's wrong with the response vector that we have above?

In [ ]:
# set all negative responses to zero...
R[R<0]=0

# plot
plt.plot(R, 'k-', linewidth=2)
plt.xlabel('Neuron Number', **fig_font)
plt.ylabel('Response', **fig_font)
plt.show()

###  Then generate the weights to apply to R. Notice that this is exactly what the cos(theta-theta_pref_i) does...and its related to the magnitude of the response. By 'weight' here I just mean that the neurons that are most tuned to the stim will have the highest SNR response and should thus be 'trusted' the most in computing the log likelihood function. 

In [ ]:
# take the part of the equation from above that computes the weight
w = np.cos(s1 - nDir) 

# plot
plt.plot(w, 'k-', linewidth=2)
plt.xlabel('Neuron Number', **fig_font)
plt.ylabel('Weight towards LL', **fig_font)
plt.show()

### Then compute the full LL for each possible stimulus in our space by looping over neuron, getting its response, and weighting it by its tuning funtion's offset to the stimulus. 

In [ ]:
# initialize an array of zeros...
logl = np.zeros(nDirs)

for s in np.arange(nDirs):
    logl[s] = k * np.sum(R*np.cos(sDir[s]-nDir))
                
# plot
plt.plot(logl, 'k-', linewidth=2)
plt.xlabel('Motion direction', **fig_font)
plt.ylabel('logL', **fig_font)
plt.show()

#MAP estimate of stim direction!
print('Estimated motion direction:', sDir[np.argmax(logl)]*180/pi, ' Degrees') 
print('Actual motion direction:', s1*180/pi, ' Degrees') 

## Now make Figure 4 from J&M 2006 (sort of) - how to do discrimination between two stims?
* first define 2 stims

In [ ]:
noise = 0        # noise (IID) of neural responses, set to 0 to make this easier to understand...
s1 = pi
s2 = pi*2 
s2 = pi+pi/16

### Now regenerate responses...

In [ ]:
# loop over all possible sigs
fx = np.zeros((N,nDirs))

# loop over neurons...each of which is tuned to nDir[i]
for i in np.arange(N): 
    fx[i,:] = np.exp(k*(np.cos(sDir-nDir[i])-1))

## We want to determine which stimulus is most likely given an observed pattern of responses. 

* One common scheme is the neuron/anti-neuron case developed by Newsome in which the likelihood of a given stimulus is a function of activity in neurons tuned to the stimulus minus the activity of neurons tuned to the opposite stimulus (the evil "anti-neurons"). This works fine, but it ignores the contribution of all neurons that aren't tuned to the taregt stimulus (or to the anti-target-stimulus)...i.e. its not an optimal model. 

* An optimal approach, or one that uses all available information, is specified by the present model. 

* What we want is to compute the ratio of the likelihood of each stimulus, so:

$$LR = L(\theta_{1}) / L(\theta_{2})$$


* Going to log space we would have:

$$logLR = \log L(\theta_{1}) - \log L(\theta_{2})$$

* Which is: 

$$logLR = k * \sum_{i=1}^{N}n_{i}[cos(\theta_{1} - \theta_{i})-cos(\theta_{2} - \theta_{i})]$$

* Or

$$LogLRatio = k * \sum_{i=1}^{N}(R_{i}[cos(\theta_{1} - \theta_{i})-cos(\theta_{2} - \theta_{i})]$$

* And 

$$cos(\theta_{1} - \theta_{i})$$

is the expected response of each neuron to s1, or the weights that we want to apply to the observed response profile to compute the likelihood of s1...

* And then we do the same for s2. 

* So the weighting function when computing the ratio of the likelihoods is the difference between the weighting functions associated with each individual stimulus. 

In [ ]:
# repeat our steps to figure out what the response profile is across neurons for the 
# stimulus that was actually presented (lets say it was s1)
sInd = np.argmin(np.abs(s1-sDir))

# pull a column out of the matrix that defines our TFs. 
R = fx[:,sInd].copy()

if noise:
    R += np.random.randn(len(R))*noise

# fix any negative responses...
R[R<0] = 0 

### Generate the weights for s1 and s2 - this is determined by the expected response of a neuron to each stimulus

In [ ]:
w1 = np.cos(s1-nDir)
w2 = np.cos(s2-nDir)

### Contribution of neurons to logL(s1) - this is the actual response of each neuron multiplied by the weight of each neuron (which is the TF, or the expected response of the neuron)

In [ ]:
logL1=R*w1

### Then contribution of each neuron to logL(s2)

In [ ]:
logL2=R*w2

### Compute contribution of each neuron to LLR


In [ ]:
logLR_cont = k * (R*(w1-w2))

### Compute the actual log likelihood ratio which is the sum of LLR for each neuron

In [ ]:
logLR = k * np.sum(R*(w1-w2))

In [ ]:
# plot the data
nAxis = nDir*180/pi;
nInd = np.argmin(np.abs(s1*(180/pi)-nAxis)) # find the stimulus index

In [ ]:
# plot...
plt.plot(nAxis, R, 'o-', linewidth=2)
plt.xlabel('Motion Direction', **fig_font)
plt.ylabel('Response', **fig_font)
plt.title('Observed response across neurons', **fig_font)
plt.show()

plt.plot(nAxis, w1, '-', linewidth=2)
plt.plot(nAxis, w2, ':', linewidth=2)
plt.xlabel('Motion Direction', **fig_font)
plt.ylabel('Weights', **fig_font)
plt.title('Weights for s1 and s2', **fig_font)
plt.show()

plt.plot(nAxis, logL1, linewidth=2)
plt.plot(nAxis, logL2, ':', linewidth=2)  
plt.xlabel('Motion Direction', **fig_font)
plt.ylabel('Contribution of each Neuron to LL', **fig_font)
plt.title('Contribution of each Neuron to LL', **fig_font)
plt.show()

plt.plot(nAxis, logL1-logL2, linewidth=2)
plt.xlabel('Motion Direction', **fig_font)
plt.ylabel('Contribution of each Neuron to LLR', **fig_font)
plt.title('Contribution of each Neuron to LLR', **fig_font)
plt.show()

